<a href="https://colab.research.google.com/github/Johann-Huber/batchnorm_pytorch/blob/main/batch_normalization_in_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Libs
import numpy as np
import torch
from torch import nn
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import matplotlib.pyplot as plt 
from tqdm.notebook import tqdm

# Seeds
torch.manual_seed(0)
np.random.seed(0)

In [ ]:
# Import the MINST dataset

transform_imgs = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),])
                               #torchvision.transforms.Normalize(
                               #  (0.1307,), (0.3081,))])

mnist_trainset = datasets.MNIST(root='./data', 
                                train=True, 
                                download=True, 
                                transform=transform_imgs)

mnist_testset = datasets.MNIST(root='./data', 
                               train=False, 
                               download=True, 
                               transform=transform_imgs)

In [ ]:
## Dataset loader


train_loader_params = {'shuffle': True, 'batch_size' : 60, 'drop_last' : False}
test_loader_params = {'shuffle': False, 'batch_size' : 60, 'drop_last' : False}

train_loader = DataLoader(mnist_trainset, **train_loader_params )
test_loader = DataLoader(mnist_testset, **test_loader_params )

# Each batch : (imgs, targets)
#    imgs : (batch_size=60, channels=1, height=28, width=28) tensor (float32)
#    targets : (batch_size=60) tensor (int64)


In [ ]:
## BatchNorm reimplementation

class myBatchNorm2d(nn.Module):
    def __init__(self, input_size = None , epsilon = 1e-3, momentum = 0.99):
        super(myBatchNorm2d, self).__init__()
        assert input_size, print('Missing input_size parameter.')
        
        # Batch mean & var must be defined during training
        self.mu = torch.zeros(1, input_size)
        self.var = torch.ones(1, input_size)
        
        # For numerical stability
        self.epsilon = epsilon
        
        # Exponential moving average for mu & var update 
        self.it_call = 0  # training iterations
        self.momentum = momentum # EMA smoothing
        
        # Trainable parameters
        self.beta = torch.nn.Parameter(torch.zeros(1, input_size))
        self.gamma = torch.nn.Parameter(torch.ones(1, input_size))
        
        # Batch size on which the normalization is computed
        self.batch_size = 0

        
    def forward(self, x):
        # [batch_size, input_size]
        
        self.it_call += 1
        
        if self.training :
            
            if( self.batch_size == 0 ):
                # First iteration : save batch_size
                self.batch_size = x.shape[0]
            
            # Training : compute BN pass
            batch_mu = (x.sum(dim=0)/x.shape[0]).unsqueeze(0) # [1, input_size]
            batch_var = (x.var(dim=0)/x.shape[0]).unsqueeze(0) # [1, input_size]
            
            x_normalized = (x-batch_mu)/torch.sqrt(batch_var + self.epsilon) # [batch_size, input_size]
            x_bn = self.gamma * x_normalized + self.beta # [batch_size, input_size]
            
            
            # Update mu & std 
            if(x.shape[0] == self.batch_size):
                running_mu = batch_mu
                running_var = batch_var
            else:
                running_mu = batch_mu*self.batch_size/x.shape[0]
                running_var = batch_var*self.batch_size/x.shape[0]
 
            self.mu = running_mu * (self.momentum/self.it_call) + \
                            self.mu * (1 - (self.momentum/self.it_call))
            self.var = running_var * (self.momentum/self.it_call) + \
                        self.var * (1 - (self.momentum/self.it_call))
            
        else:
            # Inference : compute BN pass using estimated mu & var
            if (x.shape[0] == self.batch_size):
                estimated_mu = self.mu
                estimated_var = self.var
            else :
                estimated_mu = self.mu*x.shape[0]/self.batch_size
                estimated_var = self.var*x.shape[0]/self.batch_size
                
            x_normalized = (x-estimated_mu)/torch.sqrt(estimated_var + self.epsilon) # [batch_size, input_size]
            x_bn = self.gamma * x_normalized + self.beta # [batch_size, input_size]
    
        return x_bn # [batch_size, output_size=input_size]


In [ ]:
# Write an activation tracker module
#
# Identity layer, keeping track of percentiles. Getter function with ndarray as results


## BatchNorm reimplementation

class ActivationTracker(nn.Module):
    '''Identity module, which keep track of the current activation during validation.'''
    
    def __init__(self):
        super(ActivationTracker, self).__init__()

        # Keep tack of [0.15, 0.5, 0.85] percentiles
        self.percents_activation_track = [15, 50, 85]
        self.all_percents_activation = []

    def get_all_activations(self):
        return np.array(self.all_percents_activation)
        
    def forward(self, x):
        
        if not self.training :
            percents_activation = np.percentile(x.detach().flatten(), self.percents_activation_track)
            self.all_percents_activation.append(percents_activation)
            #print('percents_activation = ', percents_activation)
        
        return x


In [ ]:
## Networks

def init_weights(model):
    for module in model:
        if type(module) == nn.Linear:
            torch.nn.init.normal_(module.weight, mean=0.0, std=1.0) # "Random Gaussian value"
            #torch.nn.init.xavier_uniform_(module.weight)
            module.bias.data.fill_(0.)


input_size = 784

# Baseline
baseline_model = nn.Sequential(nn.Linear(input_size,100), #1
                      nn.Sigmoid(),
                      nn.Linear(100,100), #2
                      nn.Sigmoid(),
                      nn.Linear(100,100), #3
                      ActivationTracker(),
                      nn.Sigmoid(),
                      nn.Linear(100,10) # out
                     )

init_weights(baseline_model)


        
# BN model
bn_model = nn.Sequential(nn.Linear(input_size,100), #1
                      myBatchNorm2d(100),
                      nn.Sigmoid(),
                      nn.Linear(100,100), #2
                      myBatchNorm2d(100),
                      nn.Sigmoid(),
                      nn.Linear(100,100), #3
                      myBatchNorm2d(100),
                      ActivationTracker(),
                      nn.Sigmoid(),
                      nn.Linear(100,10) # out
                     )

init_weights(bn_model)
    



In [ ]:
# Loss & metrics
criterion = nn.CrossEntropyLoss()
metric = accuracy_score

In [ ]:
def valid_loop(model, valid_loader, criterion, metric, epoch, verbose = True):
    
    sum_loss = 0
    sum_score = 0
    
    for it, (imgs, targets) in enumerate(valid_loader, start=1):
        imgs = imgs.view(-1,784)
        
        with torch.no_grad():
            out = model(imgs) # [batch_size,num_class]
            preds  = torch.argmax(out.detach(), dim=1) # [batch_size]

            loss = criterion(out,targets)
            score = metric(targets, preds)

                    
            sum_loss += loss
            sum_score += score
            
    return sum_score/it, sum_loss/it

In [ ]:
## Training models


def train_loop(model, train_loader, valid_loader, optimizer, scheduler, criterion, metric, verbose = True):

    # Loss & metric lists
    valid_stats = []
    epochs_valid_stats = []

    with tqdm(range(num_epochs), desc = "Train epochs") as epochs_bar :
      for e in epochs_bar:
          
          # Training
          with tqdm(train_loader, leave=False) as it_bar:
            for it, (imgs, targets) in enumerate(it_bar, start=1):
                imgs = imgs.view(-1,784)

                out = model(imgs) # [batch_size,num_class]
                preds  = torch.argmax(out.detach(), dim=1) # [batch_size]

                loss = criterion(out,targets)
                score = metric(targets, preds)

                #if(verbose):
                #    it_bar.set_postfix(train_loss=loss.item(), train_score=score)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                if(it % valid_steps == 0):
                    # Validation
                    model.eval()
                    valid_score, valid_loss = valid_loop(model, valid_loader, criterion, metric, e, verbose)
                    valid_stats.append([valid_score.astype(np.float32), \
                                        valid_loss.detach().numpy().astype(np.float32)])
                    epochs_valid_stats.append(it+e*len(train_loader))
                    
                    if(verbose):
                      it_bar.set_postfix(valid_loss=valid_loss.item(), valid_score=valid_score)

                    model.train()

            scheduler.step()

    return np.array(valid_stats), epochs_valid_stats



def init_optim_and_scheduler(model, lr = 0.1):
    #optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9) # momentum not mentioned
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10) # Not mentioned in the paper
    return optimizer, scheduler



#---------------------------
# Training hyperparameters
#---------------------------

#num_iterations = 50000
num_iterations = 10000 #10000
num_epochs = int(num_iterations/len(train_loader))

valid_steps = 50 # training iterations before validation

verbose = True
   
LR_BASE = 2.
LR_BN = 2.
    
#---------------------------
# Training loops
#---------------------------


## Without BN
print('-'*15, 'BASELINE MODEL', '-'*15)
optimizer, scheduler = init_optim_and_scheduler(baseline_model, lr = LR_BASE)
valid_stats_base, epochs_stats = train_loop(baseline_model, train_loader, test_loader, \
                                            optimizer, scheduler, criterion, metric, verbose = verbose)

## With BN
print('-'*15, 'BATCH NORMALIZED MODEL', '-'*15)
optimizer, scheduler = init_optim_and_scheduler(bn_model, lr = LR_BN)
valid_stats_bn, epochs_stats = train_loop(bn_model, train_loader, test_loader, \
                                          optimizer, scheduler, criterion, metric, verbose = verbose)




--------------- BASELINE MODEL ---------------


In [ ]:


if epochs_stats: 
    fig, ax = plt.subplots(1, 2, figsize=(10*2, 5))
    #ax.clear()
    
    # Scores
    ax[0].plot(epochs_stats, valid_stats_base[:, 0], 'k--', \
               label = f"Train (baseline) score {valid_stats_base[-1, 0]:.4f}")
    ax[0].plot(epochs_stats, valid_stats_bn[:, 0], 'b-', \
               label = f"Train (with BN) score {valid_stats_bn[-1, 0]:.4f}")
    
    # Losses
    ax[1].plot(epochs_stats, valid_stats_base[:, 1], 'k--', \
               label = f"Train (baseline) loss {valid_stats_base[-1, 1]:.4f}")
    ax[1].plot(epochs_stats, valid_stats_bn[:, 1], 'b-', \
               label = f"Train (with BN) loss {valid_stats_bn[-1, 1]:.4f}")
    
    ax[0].legend(); ax[1].legend()
    plt.show()


In [ ]:

# Get last hidden layer activation
baseline_activation = baseline_model[5].get_all_activations()
bn_activation = bn_model[8].get_all_activations()

# Reshape from [num_call_valid_bn,3] to [num_call_valid, num_test_batches, 3]
num_call_valid = len(epochs_stats)
num_test_batches = len(test_loader)
baseline_activation = np.reshape(baseline_activation, (num_call_valid,num_test_batches,3))
bn_activation = np.reshape(bn_activation, (num_call_valid,num_test_batches,3))

# Get mean over all the test batches
baseline_activation = np.mean(baseline_activation, axis=1)
bn_activation = np.mean(bn_activation, axis=1)


In [ ]:

# Display activations
if epochs_stats: 
    fig, ax = plt.subplots(1, 2, figsize=(10*2, 5))

    # -- All iterations
    # Baseline
    ax[0].plot(epochs_stats, baseline_activation[:, 0], label = f"0.15 percentile")
    ax[0].plot(epochs_stats, baseline_activation[:, 1], label = f"0.50 percentile")
    ax[0].plot(epochs_stats, baseline_activation[:, 2], label = f"0.85 percentile")
    ax[0].set_title('Without BN')
    ax[0].set_xlabel('Iterations')
    ax[0].set_ylabel('Activation (mean over the testset)')
    
    # BN based
    ax[1].plot(epochs_stats, bn_activation[:, 0], label = f"0.15 percentile")
    ax[1].plot(epochs_stats, bn_activation[:, 1], label = f"0.50 percentile")
    ax[1].plot(epochs_stats, bn_activation[:, 2], label = f"0.85 percentile")
    ax[1].set_title('With BN')
    ax[1].set_xlabel('Iterations')
    ax[1].set_ylabel('Activation (mean over the testset)')
    
    ax[0].legend(); ax[1].legend()
    plt.suptitle('Last hidden layer activations', fontsize=20)
    plt.show()
    



In [ ]:
# -- Smallest window to visualize smoothing effect
small_window = 250

if epochs_stats: 
    fig, ax = plt.subplots(1, 2, figsize=(10*2, 5))

    # Baseline
    ax[0].plot(epochs_stats[:small_window], baseline_activation[:small_window, 1], label = f"0.50 percentile")
#    ax[0].set_xlim(100, 500)
#    ax[0].set_ylim(-10, -6)
    ax[0].set_title('Without BN')
    ax[0].set_xlabel('Iterations')
    ax[0].set_ylabel('Activation (mean over the testset)')
    
    # BN based
    ax[1].plot(epochs_stats[:small_window], bn_activation[:small_window, 1], label = f"0.50 percentile")
#    ax[1].set_xlim(100, 500)
#    ax[1].set_ylim(-2, 2)
    ax[1].set_title('With BN')
    ax[1].set_xlabel('Iterations')
    ax[1].set_ylabel('Activation (mean over the testset)')
    
    ax[0].legend(); ax[1].legend()
    plt.suptitle('BN effet on activation smoothness', fontsize=20)
    plt.show()
    

